In [3]:
from bs4 import BeautifulSoup as BS
import urllib.request
import pandas as pd
import numpy as np
import re

In [4]:
def scrape_appropriations_table(url, class_, n_col):
    """Scrape table to retreive values of NIH appropriations per department"""
    
    # request access to url
    request = urllib.request.Request(url)
    result = urllib.request.urlopen(request)
    
    # read in raw html
    result_text = result.read()
    
    # use beautiful soup to parse html
    soup = BS(result_text, "html.parser")
    
    # select table
    table = soup.find_all("table", class_ = class_)[0] # text is in list of lists for each table.
                                                       # index to select first (only) one
    
    # components of table are separated by tags, select just headers
    table_headers = table.find_all("th")
    
    # remove html and take just the text
    labels = [th.text for th in table_headers]
    
    # column and row labels are combined in same list so separate for constructing df downstream
    columns = labels[1:n_col]
    columns_clean = [re.findall(r'\D+', text)[0] for text in columns] # some column values have footnotes,
                                                                      # use regex to select non-digit values
    
    index = [ind[:5] for ind in labels[n_col:]] # some index values have footnotes, index to remove them
    index_label = labels[0]
    
    # loop over rest of rows to get the data
    rows = list()
    for tr in table.find_all("tr")[1:]:
        row_data = [td.text for td in tr.find_all("td")]
        rows.append(row_data)
    
    # construct df
    df = pd.DataFrame(rows, columns = columns_clean, index = index)
    
    # clean up nan values
    df.replace("\xa0", np.nan)
    
    # add index label
    df.index.name = index_label
    
    return(df)

In [5]:
# scrape the first page
nih_aprop_1 = scrape_appropriations_table("https://www.nih.gov/about-nih/what-we-do/nih-almanac/appropriations-section-1",
                                          "table-data-tight",
                                          15)

In [6]:
# scrape the second
nih_aprop_2 = scrape_appropriations_table("https://www.nih.gov/about-nih/what-we-do/nih-almanac/appropriations-section-2",
                                          "table-data-tight",
                                          16)

In [7]:
# combine into one df
nih_budget = nih_aprop_1.join(nih_aprop_2, how = "inner")

In [8]:
# shows what all columns are 

#nih_budget.info()

In [9]:
# remove commas and convert values to numeric
for s in nih_budget:
    nih_budget[s] = nih_budget[s].str.replace(",", '')
    nih_budget[s] = pd.to_numeric(nih_budget[s], errors = "coerce")

In [10]:
nih_budget.head()

,NCI,NHLBI,NIDCR,NIDDK,NINDS,NIAID,NIGMS,NICHD,NEI,NIEHS,...,NIMHD,NCRR,NCCAM,NCATS,FIC,NLM,OD,B&F,OAR,Total
FY,,,,,,,,,,,,,,,,,,,,,
1938,400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,64.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,464
1939,400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,64.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,464
1940,570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,137.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,707
1941,570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,141.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,711
1942,565,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,135.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,700


In [11]:
nih_budget.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80 entries, 1938 to 2018
Data columns (total 29 columns):
NCI      80 non-null int64
NHLBI    68 non-null float64
NIDCR    68 non-null float64
NIDDK    64 non-null float64
NINDS    64 non-null float64
NIAID    64 non-null float64
NIGMS    52 non-null float64
NICHD    55 non-null float64
NEI      48 non-null float64
NIEHS    51 non-null float64
NIA      42 non-null float64
NIAMS    30 non-null float64
NIDCD    28 non-null float64
NIMH     41 non-null float64
NIDA     24 non-null float64
NIAAA    24 non-null float64
NINR     30 non-null float64
NHGRI    27 non-null float64
NIBIB    15 non-null float64
NIMHD    16 non-null float64
NCRR     75 non-null float64
NCCAM    17 non-null float64
NCATS    5 non-null float64
FIC      50 non-null float64
NLM      49 non-null float64
OD       48 non-null float64
B&F      47 non-null float64
OAR      1 non-null float64
Total    80 non-null int64
dtypes: float64(27), int64(2)
memory usage: 18.8+ KB


In [12]:
nih_budget.to_csv("data/nih_budget.csv")